Benchmarking integrators on statistical distributions from 3 langs. Matlab has a huge overhead

In [156]:
%%time
#### Python
import scipy.stats as spstat
import scipy.integrate as spint
f = spstat.gamma(a=1).cdf
for i in range(int(1E4)):
    b= spint.quad(f,0,1)
print b

# (0.3678794411714423, 4.08428225874771e-15)
# CPU times: user 22.9 s, sys: 0 ns, total: 22.9 s
# Wall time: 22.9 s

(0.3678794411714423, 4.08428225874771e-15)
CPU times: user 22.9 s, sys: 0 ns, total: 22.9 s
Wall time: 22.9 s


In [175]:
%%time
import scipy.stats as spstat
import scipy.integrate as spint
import scipy.special as spspec

f = lambda x: spstat.gamma._cdf(x,1)
b= spint.quad(f,0,1)
for i in range(int(1E4)):
    b= spint.quad(f,0,1)
print b

# (0.3678794411714423, 4.08428225874771e-15)
# CPU times: user 792 ms, sys: 16 ms, total: 808 ms
# Wall time: 767 ms

(0.3678794411714423, 4.08428225874771e-15)
CPU times: user 752 ms, sys: 24 ms, total: 776 ms
Wall time: 743 ms


In [177]:
p = spstat.gamma(a=1)
??p.cdf
??p.dist.cdf
def cdf(self, x, *args, **kwds):
    """
    Cumulative distribution function of the given RV.

    Parameters
    ----------
    x : array_like
        quantiles
    arg1, arg2, arg3,... : array_like
        The shape parameter(s) for the distribution (see docstring of the
        instance object for more information)
    loc : array_like, optional
        location parameter (default=0)
    scale : array_like, optional
        scale parameter (default=1)

    Returns
    -------
    cdf : ndarray
        Cumulative distribution function evaluated at `x`

    """
    args, loc, scale = self._parse_args(*args, **kwds)
    x, loc, scale = map(asarray, (x, loc, scale))
    args = tuple(map(asarray, args))
    dtyp = np.find_common_type([x.dtype, np.float64], [])
    x = np.asarray((x - loc)/scale, dtype=dtyp)
    cond0 = self._argcheck(*args) & (scale > 0)
    cond1 = self._open_support_mask(x) & (scale > 0)
    cond2 = (x >= self.b) & cond0
    cond = cond0 & cond1
    output = zeros(shape(cond), dtyp)
    place(output, (1-cond0)+np.isnan(x), self.badvalue)
    place(output, cond2, 1.0)
    if np.any(cond):  # call only if at least 1 entry
        goodargs = argsreduce(cond, *((x,)+args))
        place(output, cond, self._cdf(*goodargs)) #### ACTUAL Evaluation
    if output.ndim == 0:
        return output[()]
    return output

In [135]:
DIR = 'gammacdf'

In [154]:
%%time
### Julia
lines = ''''
using Distributions
using Cubature
D = Gamma(1)
f = x ->begin cdf(D,x);end
reps = 1E4
@time for j in 1:reps 
    b = quadgk(f,0,1)
    end
@time for j in 1:reps 
    b = hquadrature(f,0,1)
    end
b = quadgk(f,0,1)
println(b)' '''
!echo {lines} > {DIR}/test.jl
!julia {DIR}/test.jl
print 

  0.927352 seconds (1.34 M allocations: 35.109 MiB, 1.57% gc time)
  0.296866 seconds (839.09 k allocations: 23.614 MiB, 1.31% gc time)
(0.3678794411714423, 5.551115123125783e-17)

CPU times: user 108 ms, sys: 16 ms, total: 124 ms
Wall time: 5.82 s


In [152]:
%%time
### Matlab
lines = r''' "start = tic;
f = @(x)gamcdf(x,1,1);
for i = 1:1E4
    b = integral(f,0,1);
end
disp(b)
fprintf('%s elapsed: %f s ', mfilename,toc(start)) " ''' 
!echo {lines} >{DIR}/test.m
!matlab -nodisplay -r "run('gammacdf/test.m');exit;"
print '\n'


=
                            < M A T L A B (R) >
                  Copyright 1984-2016 The MathWorks, Inc.
                   R2016b (9.1.0.441655) 64-bit (glnxa64)
                             September 7, 2016

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 
    0.3679

test elapsed: 5.680399 s >

CPU times: user 272 ms, sys: 36 ms, total: 308 ms
Wall time: 13.5 s


In [184]:
%%time
### R
lines = ''''
main <- function(){
    f = function(x){pgamma(x,1)}
    reps = 1E4
    for (j in 1:reps){
        b = integrate(f,0,1)
    }
    b
}
system.time({b <- main()})
print(b)
' '''
!echo {lines} > {DIR}/test.R
!Rscript {DIR}/test.R
print 

   user  system elapsed 
  0.316   0.000   0.318 
0.3678794 with absolute error < 4.1e-15

CPU times: user 4 ms, sys: 20 ms, total: 24 ms
Wall time: 634 ms
